<a href="https://colab.research.google.com/github/ergul13/mr_akgul/blob/main/%C3%A7oklu_s%C4%B1n%C4%B1fland%C4%B1rma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

# Veri yolları
train_csv_path = '/content/drive/MyDrive/SPLIT_DATA/train.csv'
val_csv_path = '/content/drive/MyDrive/SPLIT_DATA/val.csv'
test_csv_path = '/content/drive/MyDrive/SPLIT_DATA/test.csv'

# Verileri yükle
df_train = pd.read_csv(train_csv_path)
df_val = pd.read_csv(val_csv_path) if os.path.exists(val_csv_path) else pd.DataFrame()
df_test = pd.read_csv(test_csv_path)

# Renk paleti
colors = sns.color_palette('pastel')[0:5]

# 1. Genel Dağılım (Eğitim Seti)
plt.figure(figsize=(18, 6))

# Pasta Grafiği: BI-RADS Dağılımı
plt.subplot(1, 3, 1)
counts = df_train['label'].value_counts().sort_index()
plt.pie(counts, labels=[f'BI-RADS {i}' for i in counts.index], autopct='%1.1f%%', colors=colors, startangle=140)
plt.title('Eğitim Seti Sınıf Dağılımı (BI-RADS)')

# Çubuk Grafik: Sayısal Dağılım
plt.subplot(1, 3, 2)
sns.barplot(x=counts.index, y=counts.values, palette='viridis')
plt.title('Sınıf Başına Görüntü Sayısı')
plt.xlabel('BI-RADS Sınıfları')
plt.ylabel('Adet')
for i, v in enumerate(counts.values):
    plt.text(i, v + 50, str(v), ha='center', fontweight='bold')

# Set Dağılımı (Train/Val/Test)
plt.subplot(1, 3, 3)
sets = ['Eğitim', 'Doğrulama', 'Test']
values = [len(df_train), len(df_val), len(df_test)]
sns.barplot(x=sets, y=values, palette='rocket')
plt.title('Veri Seti Ayrımı')
plt.ylabel('Toplam Görüntü')
for i, v in enumerate(values):
    plt.text(i, v + 100, str(v), ha='center')

plt.tight_layout()
plt.show()

In [ ]:
def visualize_augmentation_steps(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224))

    # İşlemler
    augmentations = {
        'Orijinal (224x224)': img,
        'Rotate 90': cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE),
        'Rotate 180': cv2.rotate(img, cv2.ROTATE_180),
        'Rotate 270': cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE),
        'Horizontal Flip': cv2.flip(img, 1)
    }

    plt.figure(figsize=(15, 3))
    plt.suptitle('Model Eğitiminde Uygulanan Veri Artırma (Augmentation) Adımları', fontsize=14, y=1.05)

    for i, (name, aug_img) in enumerate(augmentations.items()):
        plt.subplot(1, 5, i+1)
        plt.imshow(aug_img, cmap='gray')
        plt.title(name)
        plt.axis('off')
    plt.show()

# BI-RADS 4 sınıfından bir örnek seçip işlemleri uygula
sample_path = df_train[df_train['label'] == 4].iloc[0]['path']
visualize_augmentation_steps(sample_path)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

# CSV yolları
train_csv_path = '/content/drive/MyDrive/SPLIT_DATA/train.csv'
val_csv_path = '/content/drive/MyDrive/SPLIT_DATA/val.csv' # Validation dosyan varsa

# 1. CSV'leri Yükle
if os.path.exists(train_csv_path):
    df_train = pd.read_csv(train_csv_path)
    print(f"Eğitim listesi yüklendi: {len(df_train)} kayıt")
else:
    print("HATA: train.csv bulunamadı!")

if os.path.exists(val_csv_path):
    df_val = pd.read_csv(val_csv_path)
    print(f"Validation listesi yüklendi: {len(df_val)} kayıt")
else:
    print("UYARI: val.csv bulunamadı, validation boş geçilecek.")
    df_val = pd.DataFrame()

# 2. Model 3 (BI-RADS 4) İçin Veriyi Hazırla
# Hedef Sınıf: 4 (Pozitif), Diğerleri: Negatif
def prepare_data_from_csv(df, target_class=4):
    if len(df) == 0: return df

    df_pos = df[df['label'] == target_class].copy()
    df_neg = df[df['label'] != target_class].copy()

    # Pozitifler için Augmentation işaretlemesi (Sanal Çoğaltma)
    augmented_pos = []
    for _, row in df_pos.iterrows():
        # 4 Farklı varyasyon
        augmented_pos.append({'path': row['path'], 'binary_label': 1, 'aug_type': 'original'})
        augmented_pos.append({'path': row['path'], 'binary_label': 1, 'aug_type': 'rot90'})
        augmented_pos.append({'path': row['path'], 'binary_label': 1, 'aug_type': 'rot180'})
        augmented_pos.append({'path': row['path'], 'binary_label': 1, 'aug_type': 'rot270'})

    df_pos_aug = pd.DataFrame(augmented_pos)

    # Negatiflerden örneklem al (Pozitif sayısının 2 katı kadar)
    n_pos = len(df_pos_aug)
    n_neg_sample = min(len(df_neg), n_pos * 2)

    if n_neg_sample > 0:
        df_neg_sampled = resample(df_neg, n_samples=n_neg_sample, replace=False, random_state=42)
        neg_data = []
        for _, row in df_neg_sampled.iterrows():
            neg_data.append({'path': row['path'], 'binary_label': 0, 'aug_type': 'original'})
        df_neg_final = pd.DataFrame(neg_data)

        # Birleştir
        df_final = pd.concat([df_pos_aug, df_neg_final]).sample(frac=1, random_state=42).reset_index(drop=True)
        return df_final
    else:
        return df_pos_aug

# Eğitim verisini hazırla
df_model_3 = prepare_data_from_csv(df_train, target_class=4)
print(f"Model 3 Eğitim Verisi Hazır: {len(df_model_3)} adet")

# Validation verisini hazırla (Augmentation yok)
val_data_list = []
if len(df_val) > 0:
    for _, row in df_val.iterrows():
        label = 1 if row['label'] == 4 else 0
        val_data_list.append({'path': row['path'], 'binary_label': label, 'aug_type': 'original'})
    df_val_model3 = pd.DataFrame(val_data_list)
    print(f"Model 3 Validation Verisi Hazır: {len(df_val_model3)} adet")
else:
    df_val_model3 = pd.DataFrame()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import cv2
import copy
from sklearn.metrics import f1_score

# --- AYARLAR ---
BATCH_SIZE = 64 # H100 güçlü olduğu için batch size'ı artırdım
EPOCHS = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Çalışma Cihazı: {device}")

# --- SINIF TANIMLARI ---
class MammographyDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
    def __len__(self):
        return len(self.dataframe)
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img = cv2.imread(row['path'])
        if img is None:
            img = np.zeros((224, 224), dtype=np.uint8)
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (224, 224))

        # Augmentation
        aug = row['aug_type']
        if aug == 'rot90': img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif aug == 'rot180': img = cv2.rotate(img, cv2.ROTATE_180)
        elif aug == 'rot270': img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

        img = img.astype(np.float32) / 255.0
        img = np.expand_dims(img, axis=0)
        return torch.from_numpy(img), torch.tensor(row['binary_label'], dtype=torch.float32)

class Swish(nn.Module):
    def forward(self, x): return x * torch.sigmoid(x)

class MammographyModel(nn.Module):
    def __init__(self):
        super(MammographyModel, self).__init__()
        self.base = models.resnet18(weights='DEFAULT')
        self.base.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.base.fc = nn.Identity()
        self.classifier = nn.Sequential(
            nn.Linear(512, 512),
            Swish(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
    def forward(self, x): return self.classifier(self.base(x))

# --- YÜKLEME VE EĞİTİM ---
train_loader = DataLoader(MammographyDataset(df_model_3), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(MammographyDataset(df_val_model3), batch_size=BATCH_SIZE, shuffle=False)

model = MammographyModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

print("Eğitim Başlıyor...")
best_f1 = 0.0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)

    train_loss = total_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            preds = torch.sigmoid(model(inputs)) > 0.5
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    val_f1 = f1_score(all_labels, all_preds, zero_division=0)
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {train_loss:.4f} | Val F1: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'best_model_3_birads4.pth')
        print("  -> Model Kaydedildi.")

Çalışma Cihazı: cuda
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 238MB/s]


Eğitim Başlıyor...
